In [1]:
import tensorflow as tf

In [2]:
daily_sales_numbers = [21, 22, -108, 31, -1, 32, 34, 31]

tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset 

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [3]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [4]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [5]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [6]:
tf_dataset = tf_dataset.filter(lambda x: x > 0)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [7]:
tf_dataset = tf_dataset.map(lambda x: x * 72)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [8]:
tf_dataset = tf_dataset.shuffle(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1584
1512
2232
2304
2448
2232


In [9]:
for sales_batch in tf_dataset.batch(2):
    print(sales_batch.numpy())

[1512 2232]
[2304 1584]
[2448 2232]


In [10]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

tf_dataset = tf_dataset.filter(lambda x: x > 0).map(lambda y: y * 72).shuffle(2).batch(2)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 2232]
[1584 2448]
[2232 2304]


## Images

In [12]:
images_ds = tf.data.Dataset.list_files('images/*/*', shuffle = False)

In [13]:
image_count = len(images_ds)
image_count

132

In [14]:
type(images_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [15]:
for file in images_ds.take(3):
    print(file.numpy())

b'images\\cat\\.ipynb_checkpoints'
b'images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg'
b'images\\cat\\7 Foods Your Cat Can_t Eat.jpg'


In [16]:
images_ds = images_ds.shuffle(200)
for file in images_ds.take(3):
    print(file.numpy())

b'images\\dog\\Rescue turns dog with untreatable tumor....jpg'
b'images\\dog\\Science_ Talking to Your Dog Means You....jpg'
b'images\\dog\\The Cost of Owning a Dog.jpg'


In [17]:
class_names = ["cat","dog"]

In [18]:
train_size = int(image_count * 0.8)
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [19]:
len(train_ds)

105

In [20]:
len(test_ds)

27

In [21]:
import os

def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2]

In [22]:
get_label("images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg")

<tf.Tensor: shape=(), dtype=string, numpy=b'cat'>

In [23]:
def process_image(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path) # load the raw data from the file as a string
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img, [128, 128])
    return img, label

In [24]:
img, label = process_image("images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg")
img.numpy()[: 2]

array([[[155.      , 186.      , 215.      ],
        [156.      , 187.      , 216.      ],
        [158.      , 189.      , 218.      ],
        [160.0039  , 189.0039  , 219.0039  ],
        [161.0039  , 190.0039  , 220.0039  ],
        [162.      , 191.      , 221.      ],
        [166.      , 193.      , 222.      ],
        [167.      , 194.      , 223.      ],
        [168.      , 195.      , 224.      ],
        [169.      , 196.      , 225.      ],
        [170.      , 197.      , 224.      ],
        [170.0039  , 197.0039  , 224.0039  ],
        [172.      , 199.      , 226.      ],
        [173.0039  , 199.0039  , 224.0039  ],
        [174.0039  , 200.0039  , 225.0039  ],
        [175.0039  , 201.0039  , 226.0039  ],
        [176.      , 202.      , 227.      ],
        [177.0039  , 203.0039  , 228.0039  ],
        [177.0039  , 203.0039  , 228.0039  ],
        [179.0039  , 203.0039  , 227.0039  ],
        [180.0039  , 204.0039  , 228.0039  ],
        [180.0039  , 204.0039  , 2

In [25]:
train_ds = train_ds.map(process_image)
test_ds = test_ds.map(process_image)

In [26]:
def scale(image, label):
    return image/255, label

In [27]:
train_ds = train_ds.map(scale)

In [28]:
for image, label in train_ds.take(5):
    print("****Image: ",image.numpy()[0][0])
    print("****Label: ",label.numpy())

****Image:  [0.5058824  0.5764706  0.24705882]
****Label:  b'cat'
****Image:  [0.32156864 0.32156864 0.32156864 1.        ]
****Label:  b'cat'
****Image:  [0.44313726 0.28627452 0.04705882]
****Label:  b'dog'
****Image:  [0.10422315 0.179294   0.07285061]
****Label:  b'dog'
****Image:  [0.2784314  0.4392157  0.29803923 0.        ]
****Label:  b'cat'


# Reviews

In [30]:
reviews_ds = tf.data.Dataset.list_files('reviews/*/*', shuffle = False)

In [31]:
review_count = len(reviews_ds)
review_count

6

In [32]:
type(reviews_ds)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [33]:
for file in reviews_ds:
    print(file.numpy())

b'reviews\\negative\\neg_1.txt'
b'reviews\\negative\\neg_2.txt'
b'reviews\\negative\\neg_3.txt'
b'reviews\\positive\\pos_1.txt'
b'reviews\\positive\\pos_2.txt'
b'reviews\\positive\\pos_3.txt'


## Extract review text from these files. Extract label from folder name


In [35]:
import os

def extract_review_and_label(file_path):
    return tf.io.read_file(file_path), tf.strings.split(file_path, os.path.sep)[-2]

In [36]:
reviews_ds_1 = reviews_ds.map(extract_review_and_label)

for review, label in reviews_ds_1:
    print("Review : ", review.numpy()[: 50])
    print("Label : ", label.numpy())

Review :  b"Basically there's a family where a little boy (Jak"
Label :  b'negative'
Review :  b'This show was an amazing, fresh & innovative idea '
Label :  b'negative'
Review :  b''
Label :  b'negative'
Review :  b'One of the other reviewers has mentioned that afte'
Label :  b'positive'
Review :  b'A wonderful little production. <br /><br />The fil'
Label :  b'positive'
Review :  b''
Label :  b'positive'


## Filter blank reviews


In [38]:
reviews_ds_2 = reviews_ds_1.filter(lambda review, label : review != "")

for review, label in reviews_ds_2.as_numpy_iterator():
    print("Review : ", review[: 50])
    print("Label : ", label)

Review :  b"Basically there's a family where a little boy (Jak"
Label :  b'negative'
Review :  b'This show was an amazing, fresh & innovative idea '
Label :  b'negative'
Review :  b'One of the other reviewers has mentioned that afte'
Label :  b'positive'
Review :  b'A wonderful little production. <br /><br />The fil'
Label :  b'positive'


## Perform map, filter and shuffle all in single line of code


In [40]:
final_ds = reviews_ds.map(extract_review_and_label).filter(lambda review, label : review != "").shuffle(3)

for review, label in final_ds.as_numpy_iterator():
    print("Review :", review[: 50])
    print("Label :", label)

Review : b'This show was an amazing, fresh & innovative idea '
Label : b'negative'
Review : b'One of the other reviewers has mentioned that afte'
Label : b'positive'
Review : b"Basically there's a family where a little boy (Jak"
Label : b'negative'
Review : b'A wonderful little production. <br /><br />The fil'
Label : b'positive'
